In [1]:
import os
import re
from xml.dom import minidom
import urllib

In [2]:
def get_correct_path(url):
    if url.startswith('/'):
        return url[1:]
    return url

In [3]:
def get_granules_xml(target_dir, file_mask, logger=None):
    # compile regexp
    mask_filter = re.compile(file_mask).match
    granules = xml_walk(target_dir)

    granule_list = map(
        lambda x: granules[x], filter(mask_filter, granules)
    )
    return granule_list

In [4]:
def xml_walk(catalog):
    catalogxml = os.path.join(catalog, 'catalog.xml')
    xmldoc = minidom.parse(urllib.urlopen(catalogxml))
    cataloglist = xmldoc.getElementsByTagName('thredds:catalogRef')
    itemlist = xmldoc.getElementsByTagName('thredds:dataset')

    granule_list = {}

    for item in itemlist:
        item_name = item.attributes['name'].value

        dataset_path = get_correct_path(
            item.attributes['ID'].value
        )
        item_url = os.path.join(catalog_url,
                                dataset_path)
        granule_list[item_name] = item_url

    granule_sublist = {}
    for subcatalog in cataloglist:
        subcatalog_path = get_correct_path(
            subcatalog.attributes['ID'].value
        )
        subcatalog_full_path = os.path.join(catalog_url,
                                            subcatalog_path)
        granule_sublist.update(
            xml_walk(subcatalog_full_path)
        )

    granule_list.update(granule_sublist)
    return granule_list
    print granule_list


In [12]:
file_mask = '^ascat_\d{8}_\d{2}\d{2}\d{2}.*\.gz$'
t_dir = 'http://10.170.0.172:8080/opendap/allData/ASCAT-L2-Coastal/'
catalog_url = 'http://10.170.0.172:8080'

{'NC_GLOBAL': {'comment': 'Orbit period and inclination are constant values. All wind directions in oceanographic convention (0 deg. flowing North)', 'orbit_number': 25051, 'creation_time': '02:18:09', 'creation_date': '2011-08-18', 'stop_date': '2011-08-18', 'processing_type': 'O', 'contents': 'ovw', 'rev_orbit_period': '6081.7', 'title': 'MetOp-A ASCAT Level 2 Coastal Ocean Surface Wind Vector Product', 'source': 'MetOp-A ASCAT', 'stop_time': '01:11:58', 'service_type': 'eps', 'start_date': '2011-08-17', 'processing_level': 'L2', 'title_short_name': 'ASCAT-L2-Coastal', 'pixel_size_on_horizontal': '12.5 km', 'start_time': '23:30:00', 'software_identification_level_1': 703, 'granule_name': 'ascat_20110817_233000_metopa_25051_eps_o_coa_2100_ovw.l2.nc', 'references': 'ASCAT Wind Product User Manual, http://www.osi-saf.org/, http://www.knmi.nl/scatterometer/', 'instrument_calibration_version': 0, 'institution': 'EUMETSAT/OSI SAF/KNMI', 'orbit_inclination': '98.7', 'equator_crossing_date':

In [13]:
granule_list = get_granules_xml(t_dir, file_mask)
#print granule_list



{'NC_GLOBAL': {'comment': 'Orbit period and inclination are constant values. All wind directions in oceanographic convention (0 deg. flowing North)', 'orbit_number': 24504, 'creation_time': '14:13:53', 'creation_date': '2011-07-10', 'stop_date': '2011-07-10', 'processing_type': 'O', 'contents': 'ovw', 'rev_orbit_period': '6081.7', 'title': 'MetOp-A ASCAT Level 2 Coastal Ocean Surface Wind Vector Product', 'source': 'MetOp-A ASCAT', 'stop_time': '13:05:58', 'service_type': 'eps', 'start_date': '2011-07-10', 'processing_level': 'L2', 'title_short_name': 'ASCAT-L2-Coastal', 'pixel_size_on_horizontal': '12.5 km', 'start_time': '11:27:00', 'software_identification_level_1': 703, 'granule_name': 'ascat_20110710_112700_metopa_24504_eps_o_coa_2000_ovw.l2.nc', 'references': 'ASCAT Wind Product User Manual, http://www.osi-saf.org/, http://www.knmi.nl/scatterometer/', 'instrument_calibration_version': 0, 'institution': 'EUMETSAT/OSI SAF/KNMI', 'orbit_inclination': '98.7', 'equator_crossing_date

In [14]:
from pydap import client
def read_attrs(granule_list):
    attrs_list = []
    for granule in granule_list:
        #print granule
        ds = client.open_url(granule)
        attrs = ds.attributes
        attrs_list.append(attrs)
        print attrs
        #print attrs_list







{'NC_GLOBAL': {'comment': 'Orbit period and inclination are constant values. All wind directions in oceanographic convention (0 deg. flowing North)', 'orbit_number': 21978, 'creation_time': '10:06:36', 'creation_date': '2011-06-07', 'stop_date': '2011-01-13', 'processing_type': 'O', 'contents': 'ovw', 'rev_orbit_period': '6081.7', 'title': 'MetOp-A ASCAT Level 2 Coastal Ocean Surface Wind Vector Product', 'source': 'MetOp-A ASCAT', 'stop_time': '17:53:58', 'service_type': 'eps', 'start_date': '2011-01-13', 'processing_level': 'L2', 'title_short_name': 'ASCAT-L2-Coastal', 'pixel_size_on_horizontal': '12.5 km', 'start_time': '16:12:00', 'software_identification_level_1': 703, 'granule_name': 'ascat_20110113_161200_metopa_21978_eps_o_coa_2000_ovw.l2.nc', 'references': 'ASCAT Wind Product User Manual, http://www.osi-saf.org/, http://www.knmi.nl/scatterometer/', 'instrument_calibration_version': 0, 'institution': 'EUMETSAT/OSI SAF/KNMI', 'orbit_inclination': '98.7', 'equator_crossing_

In [15]:
def iter_granules(granule_list, granule_count):
    '''
    Return list of list granules.
    Each sub-list consists of granule_count granules
    [[1,..,N],[1,..,N],..,[1,..,N]]

    :param granule_list:
    :param granule_count:
    '''

    return [
        list(x) for x in
        izip_longest(*[iter(granule_list)] * granule_count)
    ]
    print [
        list(x) for x in
        izip_longest(*[iter(granule_list)] * granule_count)
    ]

{'NC_GLOBAL': {'comment': 'Orbit period and inclination are constant values. All wind directions in oceanographic convention (0 deg. flowing North)', 'orbit_number': 24697, 'creation_time': '04:18:22', 'creation_date': '2011-07-24', 'stop_date': '2011-07-24', 'processing_type': 'O', 'contents': 'ovw', 'rev_orbit_period': '6081.7', 'title': 'MetOp-A ASCAT Level 2 Coastal Ocean Surface Wind Vector Product', 'source': 'MetOp-A ASCAT', 'stop_time': '03:08:59', 'service_type': 'eps', 'start_date': '2011-07-24', 'processing_level': 'L2', 'title_short_name': 'ASCAT-L2-Coastal', 'pixel_size_on_horizontal': '12.5 km', 'start_time': '01:27:01', 'software_identification_level_1': 703, 'granule_name': 'ascat_20110724_012701_metopa_24697_eps_o_coa_2000_ovw.l2.nc', 'references': 'ASCAT Wind Product User Manual, http://www.osi-saf.org/, http://www.knmi.nl/scatterometer/', 'instrument_calibration_version': 0, 'institution': 'EUMETSAT/OSI SAF/KNMI', 'orbit_inclination': '98.7', 'equator_crossing_date':

In [9]:
product_id='ASCAT-L2-Coastal'
#logger = 'None'

In [16]:
from itertools import repeat, izip_longest
from pathos.multiprocessing import ProcessingPool as Pool, cpu_count

process_count = cpu_count()-2
#print process_count
print len(granule_list)
granule_count = len(granule_list)/process_count+1
#print granule_count
pool = Pool(processes=process_count)

print read_attrs

pool.map(read_attrs,iter_granules(granule_list, granule_count+1))


28432
<function read_attrs at 0x7fa80276ede8>


{'NC_GLOBAL': {'comment': 'Orbit period and inclination are constant values. All wind directions in oceanographic convention (0 deg. flowing North)', 'orbit_number': 26680, 'creation_time': '18:46:25', 'creation_date': '2011-12-10', 'stop_date': '2011-12-10', 'processing_type': 'O', 'contents': 'ovw', 'rev_orbit_period': '6081.7', 'title': 'MetOp-A ASCAT Level 2 Coastal Ocean Surface Wind Vector Product', 'source': 'MetOp-A ASCAT', 'stop_time': '17:05:58', 'service_type': 'eps', 'start_date': '2011-12-10', 'processing_level': 'L2', 'title_short_name': 'ASCAT-L2-Coastal', 'pixel_size_on_horizontal': '12.5 km', 'start_time': '15:24:01', 'software_identification_level_1': 704, 'granule_name': 'ascat_20111210_152401_metopa_26680_eps_o_coa_2100_ovw.l2.nc', 'references': 'ASCAT Wind Product User Manual, http://www.osi-saf.org/, http://www.knmi.nl/scatterometer/', 'instrument_calibration_version': 0, 'institution': 'EUMETSAT/OSI SAF/KNMI', 'orbi

Exception: Unable to parse token: Error { 






{'NC_GLOBAL': {'comment': 'Orbit period and inclination are constant values. All wind directions in oceanographic convention (0 deg. flowing North)', 'orbit_number': 26324, 'creation_time': '17:22:10', 'creation_date': '2011-11-15', 'stop_date': '2011-11-15', 'processing_type': 'O', 'contents': 'ovw', 'rev_orbit_period': '6081.7', 'title': 'MetOp-A ASCAT Level 2 Coastal Ocean Surface Wind Vector Product', 'source': 'MetOp-A ASCAT', 'stop_time': '15:41:58', 'service_type': 'eps', 'start_date': '2011-11-15', 'processing_level': 'L2', 'title_short_name': 'ASCAT-L2-Coastal', 'pixel_size_on_horizontal': '12.5 km', 'start_time': '14:00:01', 'software_identification_level_1': 704, 'granule_name': 'ascat_20111115_140001_metopa_26324_eps_o_coa_2100_ovw.l2.nc', 'references': 'ASCAT Wind Product User Manual, http://www.osi-saf.org/, http://www.knmi.nl/scatterometer/', 'instrument_calibration_version': 0, 'institution': 'EUMETSAT/OSI SAF/KNMI', 'orbit_inclination': '98.7', 'equator_crossing_da